In [54]:
import numpy as np
import pandas as pd

In [55]:
credits = pd.read_csv('/content/drive/MyDrive/Movie Recommender System/tmdb_5000_credits.csv')
movies = pd.read_csv('/content/drive/MyDrive/Movie Recommender System/tmdb_5000_movies.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()

DATA PREPROCESSING






In [57]:
movies = movies.merge(credits, on='title')

In [58]:
movies = movies[['movie_id',  'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies.isnull().sum()

In [60]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [62]:
import ast

In [63]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
       L.append(i['name'])
       return L

In [64]:
movies['genres'] = movies['genres'].apply(convert)

In [65]:
movies['keywords'] = movies['keywords'].apply(convert)

In [66]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [67]:
movies['cast'] = movies['cast'].apply(convert3)

In [68]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
          if i['job'] == 'Director':
              L.append(i['name'])
              break
    return L

In [69]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [70]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [72]:
# Replace missing values with empty lists
movies['genres'] = movies['genres'].fillna('').apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].fillna('').apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].fillna('').apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].fillna('').apply(lambda x: [i.replace(" ", "") for i in x])


In [73]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


In [74]:
new_df = movies[['movie_id' , 'title' , 'tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df['tags']  = new_df['tags'].apply(lambda x: x.lower())

Vectorization

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words ='english')

In [79]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [81]:
import nltk

In [82]:
from nltk.stem.porter import PorterStemmer

In [83]:
ps = PorterStemmer()

In [84]:
def  stem(text):
        y  = []
        for i in text.split():
             y.append( ps.stem(i))
        return " ".join(y)

In [ ]:
new_df['tags'].apply(stem)

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

In [89]:
similarity = cosine_similarity(vectors)

In [90]:
def recommend(movie):
      movie_index = new_df[new_df['title'] == movie].index[0]
      distances = similarity[movie_index]
      movies_list = sorted(list(enumerate(distances)), reverse = True, key=lambda x:x[1])[1:6]
      for i in movie_list:
        print(new_def.iloc[i[0]].title)

In [91]:
import pickle

In [93]:
pickle.dump(new_df.to_dict(), open('movies_dict.pkl','wb'))


In [94]:
pickle.dump(similarity, open('similarity.pkl','wb'))